In [1]:
from pyneon import Recording

In [2]:
from pathlib import Path

native_dir = Path(
    r"C:\Users\qian.chu\OneDrive - University of Toronto\code_tests\Neon full native"
)
cloud_dir = Path(r"../../data/cloud_export_test")

In [3]:
rec = Recording(native_dir)

In [4]:
rec.export_cloud_format(cloud_dir, rebase=False)

C:\Users\qian.chu\Documents\GitHub\PyNeon\pyneon\video\video.py:45: UserWarning: Video info is empty and will be loaded from default values.
  warn("Video info is empty and will be loaded from default values.")


In [5]:
# rec.export_cloud_format("../../data/cloud_export_test", rebase=True)

In [6]:
# rec.export_cloud_format("../../data/cloud_export_test", rebase=True)